# Brain Wide Map: Basic Usage Example

This notebook demonstrates basic usage of the Brain Wide Map data science utilities.

In [ ]:
# Import the utilities
import sys
sys.path.append('..')

from brainwidemap import DataLoader, Explorer, Statistics, Visualizer
import numpy as np
import matplotlib.pyplot as plt

## 1. Initialize Data Loader

Connect to the IBL ONE-api to access Brain Wide Map data.

In [ ]:
# Initialize data loader
# This will attempt to connect to the remote database
# If unavailable, it will use local cache
loader = DataLoader(mode='auto')
print(f"Connected in {loader.mode} mode")

## 2. Explore Available Sessions

List and filter available experimental sessions.

In [ ]:
# Create explorer
explorer = Explorer(loader)

# List sessions with at least 400 trials
sessions = explorer.list_sessions(n_trials_min=400)
print(f"Found {len(sessions)} sessions with at least 400 trials")
sessions.head()

## 3. Load Neural Data

Load spike data from a session.

In [ ]:
# Select first session
if len(sessions) > 0:
    eid = sessions.iloc[0]['eid']
    print(f"Loading session: {eid}")
    
    # Load spike data
    spikes, clusters = loader.load_spike_data(eid)
    
    print(f"Loaded {len(spikes['times'])} spikes from {len(clusters)} units")
    print(f"Recording duration: {spikes['times'].max():.2f} seconds")

## 4. Compute Statistics

Calculate firing rates and other statistics.

In [ ]:
# Create statistics analyzer
stats = Statistics()

# Compute firing rates
firing_rates = stats.compute_firing_rates(spikes, clusters)

print(f"Mean firing rate: {firing_rates['firing_rate'].mean():.2f} Hz")
print(f"Median firing rate: {firing_rates['firing_rate'].median():.2f} Hz")

firing_rates.head()

## 5. Visualize Results

Create visualizations of the neural data.

In [ ]:
# Create visualizer
viz = Visualizer()

# Plot firing rates by brain region
if 'acronym' in firing_rates.columns or 'brain_region' in firing_rates.columns:
    fig = viz.plot_firing_rates_by_region(firing_rates, top_n=15)
    plt.show()
else:
    print("No brain region information available")

In [ ]:
# Create a raster plot for the first 10 units
# Show first 10 seconds
fig = viz.plot_raster(
    spikes['times'], 
    spikes['clusters'],
    cluster_ids=clusters.index[:10].tolist(),
    time_range=(0, 10)
)
plt.show()

## 6. Brain Region Analysis

Explore brain region coverage in this session.

In [ ]:
# Get brain region coverage
region_coverage = explorer.get_brain_region_coverage(eid)
print(f"\nBrain regions recorded: {len(region_coverage)}")
region_coverage.head(10)

In [ ]:
# Plot brain region distribution
fig = viz.plot_brain_region_distribution(clusters, top_n=15)
plt.show()

## 7. Population Statistics

Compute population-level statistics.

In [ ]:
# Compute population statistics for all units
pop_stats = stats.compute_population_statistics(spikes, clusters)

print("\nPopulation Statistics:")
for key, value in pop_stats.items():
    if isinstance(value, (int, float)):
        print(f"  {key}: {value:.3f}")
    else:
        print(f"  {key}: {value}")

## 8. Load Behavioral Data

Load and analyze behavioral performance.

In [ ]:
# Load trials
trials = loader.load_trials(eid)

print(f"\nLoaded {len(trials)} trials")
print(f"Columns: {trials.columns.tolist()}")

trials.head()

In [ ]:
# Plot behavioral performance
if 'feedbackType' in trials.columns or 'correct' in trials.columns:
    fig = viz.plot_behavioral_performance(trials, window_size=50)
    plt.show()
else:
    print("No performance information available")

## Summary

This notebook demonstrated:
- Loading Brain Wide Map data
- Computing firing rate statistics
- Visualizing neural activity
- Analyzing brain region coverage
- Examining behavioral performance

For more advanced analyses, see the other example notebooks.